In [9]:
class RankingSystem:
    def __init__(self, num_features, feature_weights, objects_features):
        # Инициализация системы ранжирования с заданным количеством признаков,
        # весами признаков и характеристиками объектов.
        self.num_features = num_features  # Количество признаков
        self.feature_weights = feature_weights  # Веса для каждого признака
        self.objects_features = objects_features  # Характеристики объектов
        # Вычисляем релевантность для каждого объекта и сохраняем в списке
        self.relevance_scores = [self.calculate_relevance(object_idx) for object_idx in range(len(objects_features))]
        # Сортируем индексы объектов по их релевантности
        self.sorted_indices = self.sort_indices()

    def calculate_relevance(self, object_idx):
        # Вычисляем релевантность объекта по его характеристикам и весам признаков
        return sum(self.feature_weights[feature_idx] * self.objects_features[object_idx][feature_idx] for feature_idx in range(self.num_features))

    def sort_indices(self):
        # Сортируем индексы объектов по их релевантности
        indices = list(range(len(self.relevance_scores)))  # Создаем список индексов объектов
        return sorted(indices, key=lambda idx: self.relevance_scores[idx])  # Сортируем индексы по значениям релевантности

    def update_feature(self, object_idx, feature_idx, new_value):
        # Обновляем значение признака для указанного объекта
        self.objects_features[object_idx][feature_idx] = new_value
        new_relevance = self.calculate_relevance(object_idx)  # Вычисляем новую релевантность

        # Обновляем релевантность объекта в списке
        old_relevance = self.relevance_scores[object_idx]
        self.relevance_scores[object_idx] = new_relevance

        # Удаляем старый индекс из отсортированного списка и вставляем новый
        self.sorted_indices.remove(object_idx)  # Удаляем старый индекс
        self.insert_sorted(object_idx)  # Вставляем индекс на новое место

    def insert_sorted(self, object_idx):
        # Вставляем индекс объекта в отсортированный список на правильную позицию
        relevance = self.relevance_scores[object_idx]  # Получаем релевантность объекта
        inserted = False  # Флаг для отслеживания вставки

        for i in range(len(self.sorted_indices)):
            if self.relevance_scores[self.sorted_indices[i]] < relevance:  # Находим позицию для вставки
                self.sorted_indices.insert(i, object_idx)  # Вставляем индекс
                inserted = True  # Устанавливаем флаг вставки в True
                break

        if not inserted:
            self.sorted_indices.append(object_idx)  # Если не вставили, добавляем в конец

    def get_top_k(self, k):
        # Получаем топ k объектов с наибольшими значениями релевантности
        if k > len(self.relevance_scores):  # Проверяем, не превышает ли k количество объектов
            k = len(self.relevance_scores)

        top_k_indices = self.sorted_indices[-k:]  # Получаем последние k индексов из отсортированного списка
        return [idx + 1 for idx in top_k_indices]  # Возвращаем индексы с учетом 1-индексации

def main():
    num_features = 2  # Количество признаков
    feature_weights = [1, 100]  # Веса признаков
    num_objects = 10  # Количество объектов
    objects_features = [  # Характеристики объектов
        [1, 2],
        [2, 1],
        [3, 1],
        [4, 1],
        [5, 1],
        [6, 1],
        [7, 1],
        [8, 1],
        [9, 1],
        [10, 1]
    ]
    num_queries = 4  # Количество запросов
    queries = [  # Запросы к системе
        "1 2",   # Получить топ-2 объекта
        "1 10",  # Получить топ-10 объектов
        "2 4 1 1000",  # Обновить признак объекта (4-й) на новое значение (1000) для первого признака
        "1 10"   # Получить топ-10 объектов после обновления
    ]

    ranking_system = RankingSystem(num_features, feature_weights, objects_features)  # Создаем экземпляр системы ранжирования
    results = []  # Список для хранения результатов запросов
    for query in queries:
        query_parts = list(map(int, query.split()))  # Разделяем запрос на части и преобразуем в целые числа
        if query_parts[0] == 1:
            top_k = query_parts[1]
            results.append(" ".join(map(str, ranking_system.get_top_k(top_k))))  # Получаем топ-k и добавляем в результаты
        elif query_parts[0] == 2:
            object_idx, feature_idx, new_value = query_parts[1] - 1, query_parts[2] - 1, query_parts[3]
            ranking_system.update_feature(object_idx, feature_idx, new_value)  # Обновляем признак объекта

    for result in results:
        print(result)  # Выводим результаты всех запросов

if __name__ == "__main__":
    main()  # Запускаем главную функцию при выполнении скрипта


10 1
2 3 4 5 6 7 8 9 10 1
4 2 3 5 6 7 8 9 10 1
